In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [51]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera'in info.get_text(): #Exclude 'Primera Publicacion'
            anuncio_item = info.get_text()
            anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
            anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
            #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [17]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [31]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [32]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
#description_list = get_job_description(result_page)
#empresa_name = get_name_empresa(result_page)
#output = unify_lists([empresa_name, anuncio_list, description_list])

In [52]:
get_job_info(trabajo_link)

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 'Tiempo Completo',
 '19 Diciembre 2018',
 '18 Enero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a

In [53]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job_description + "&page=1&listings_per_page=100"
    
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        #trabajo_list = list()
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_link)
         #   trabajo_list.append((trabajo_name,trabajo_link))
        #for trabajo in trabajo_list:
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
        #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [54]:
Results = trabajopolis("tesorería")

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/689938/Jefe-de-Contabilidad-y-Tesorería.html?searchId=1546386359.7&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1546386359.7&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/694359/Analista-de-Tesorería---La-Paz.html?searchId=1546386359.7&page=1


In [55]:
import csv
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/trabajo.csv", 'w',newline='')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        for cell in Results[i][0]:
            #print(cell)
            csvRow.append(cell)
        csvRow.append(Results[i][1])
        csvRow.append(Results[i][2])
        print(csvRow)
        writer.writerow(csvRow)
finally:
    outfile.close() 

['Importante Empresa Industrial', '689938', 'Santa Cruz de la Sierra, Bolivia', 'Contabilidad y Economía ', 'No Declarado ', 'Tiempo Completo', '21 Diciembre 2018', '20 Enero 2019', 'jefe de contabilidad y tesoreriaobjetivo del cargogestionar los procesos contables de la empresa a fin de preparar informacion financiera y economica confiable y oportuna asi como dar solucion a problemas concernientes a la elaboracion control y administracion de dicha informacion de acuerdo con principios tecnicos normas legales y tributariasprincipales funciones preparar estados financieros analizar y recomendar mejoras a los diferentes procesos que se desarrollan en la empresa con base en las evaluaciones economicasfinancieras preparar y emitir comprobantes de egresos ingresos y traspasos de acuerdo con la realidad economica de las transacciones realizadas  autorizar provisiones para el pago de obligaciones y revisar los pagos realizados  preparar conciliaciones bancarias preparar las liquidaciones de o

In [20]:
a = '\nVencimiento: 18 Enero 2019\n\n'

In [21]:
a.replace('\n','')

'Vencimiento: 18 Enero 2019'

In [22]:
trabajo_link = 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/689938/Jefe-de-Contabilidad-y-Tesorer%C3%ADa.html?searchId=1546384098.52&page=1'
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')

In [49]:
anuncio_list = []
for info in result_page.find_all('div', class_='smallListingInfo'): 
    anuncio_item = info.get_text()
    anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
    anuncio_list.append(anuncio_item.replace('\n',''))
    print(info.get_text())

ID Empleo: 695472
Ubicación: Santa Cruz de la Sierra, Bolivia
Categoría: Contabilidad y Economía 
Sueldo: De Bs. 3.000 a Bs. 3.999 
Contrato: Tiempo Completo

Publicado: 19 Diciembre 2018


Vencimiento: 18 Enero 2019




In [50]:
anuncio_list

['695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 'Tiempo Completo',
 '19 Diciembre 2018',
 '18 Enero 2019']